In [ ]:
%%bash
pip install pandas-profiling

In [1]:
import os
from glob import glob
import pandas as pd
from pandas_profiling import ProfileReport
import csv
from config import REGION, BUCKET, PROJECT, DELIM, RAW_DATA_COLS, RENAMED_COLS, LABEL_COL, STRING_COLS, NUMERIC_COLS

import sys
reload(sys)
sys.setdefaultencoding('utf8')

%matplotlib inline
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# Cloud Setup
This section is only required if running on cloud

In [ ]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Data Profiling
Data profiling is done to better understand the data, and to see if there are any invalid data (e.g. out of bounds data, unexpected data types). No data preprocessing should be done here; it should be done in tf.transform so as to have a consistent data pipeline.

In [2]:
files = glob('data/raw/Raw Data for COA classification2*.csv')
df = pd.concat([pd.read_csv(
    f, usecols=RAW_DATA_COLS, quoting=csv.QUOTE_ALL, sep=',', encoding='utf-16'
    ) for f in files], ignore_index=True)
df.columns = RENAMED_COLS
df['payment_voucher_amt'] = df['payment_voucher_amt'] \
    .str.replace(r'\(([\d,\.]+)\)', r'-\1') \
    .str.replace(',', '') \
    .astype(float)
df

,fiscal_year,business_unit,acc_code,acc_descr,voucher_id,voucher_descr,voucher_origin,vendor_id,vendor_name,voucher_line,voucher_line_descr,voucher_line_long_descr,payment_voucher_amt
0,2016,CCY,151101,Perm:Basic Salary,13205,MARCH 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,6007.0
1,2016,CCY,151101,Perm:Basic Salary,13206,MARCH 2016\n'P' D - We would like to change th...,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,5124.0
2,2016,CCY,151101,Perm:Basic Salary,13207,MARCH 2016\n'P' D - We would like to change th...,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,7655.0
3,2016,CCY,151101,Perm:Basic Salary,13633,APRIL 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,APRIL 2016,APRIL 2016,7817.0
4,2016,CCY,151101,Perm:Basic Salary,13634,APRIL 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,APRIL 2016,APRIL 2016,5511.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506890,2015,PMO,J19903,Misc Receipts/Refunds (E),00006430,CORPORATE CREDIT CARD PAYMENT FOR JUL 15,EIN,196800306E,DBS BANK LTD.,1,YEO HON SERN - 479134100004174,YEO HON SERN - 4791341000041742,-422.7
1506891,2015,PMO,J19903,Misc Receipts/Refunds (E),00013058,Gebiz Discount for POPMOEDEEPO15000003,EIN,200000251C,STAMFORD CATERING SERVICES PTE LTD,1,Gebiz Discount,Gebiz Discount,-384.0
1506892,2015,PMO,J19904,Misc Receipts/Refunds(Z),00002948,Passenger Name : MOH/JIEHUI JOAN MS Ticket No...,EIN,197600773W,PACIFIC ARENA PTE. LTD.,1,REFUND - RF032953,REFUND - RF032953,-667.7
1506893,2015,PMO,J19904,Misc Receipts/Refunds(Z),00008782,Passenger Name : LEE/MING YANG MEXCEL MR Tick...,EIN,197600773W,PACIFIC ARENA PTE. LTD.,1,REFUND - RF033622,REFUND - RF033622,-2131.8


In [15]:
acc_code_freq = df.groupby('acc_code').size().rename('count').reset_index()
acc_codes_to_include = list(acc_code_freq[acc_code_freq['count'] >= 50].acc_code)
df = df[df['acc_code'].isin(acc_codes_to_include)]
df

,fiscal_year,business_unit,acc_code,acc_descr,voucher_id,voucher_descr,voucher_origin,vendor_id,vendor_name,voucher_line,voucher_line_descr,voucher_line_long_descr,payment_voucher_amt
0,2016,CCY,151101,Perm:Basic Salary,13205,MARCH 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,6007.0
1,2016,CCY,151101,Perm:Basic Salary,13206,MARCH 2016\n'P' D - We would like to change th...,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,5124.0
2,2016,CCY,151101,Perm:Basic Salary,13207,MARCH 2016\n'P' D - We would like to change th...,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,MARCH 2016,MARCH 2016,7655.0
3,2016,CCY,151101,Perm:Basic Salary,13633,APRIL 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,APRIL 2016,APRIL 2016,7817.0
4,2016,CCY,151101,Perm:Basic Salary,13634,APRIL 2016,EIN,T08GB0028L,MAJLIS UGAMA ISLAM SINGAPURA,1,APRIL 2016,APRIL 2016,5511.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506885,2015,PMO,J19901,Misc Receipts/Refunds (T),00021153,,ONL,T08GB0009H,CIVIL SERVICE COLLEGE,1,Credit Note-TC0000002068,,-225.0
1506886,2015,PMO,J19901,Misc Receipts/Refunds (T),00021154,,ONL,T08GB0009H,CIVIL SERVICE COLLEGE,1,Credit Note-TC0000002073,,-180.0
1506887,2015,PMO,J19901,Misc Receipts/Refunds (T),00021155,,ONL,T08GB0009H,CIVIL SERVICE COLLEGE,1,Credit Note-TC0000002074,,-270.0
1506888,2015,PMO,J19902,Misc Receipts/Refunds (O),00008657,Passenger Name : CHAN/LONG HEI ROYCE MS Ticke...,EIN,197600773W,PACIFIC ARENA PTE. LTD.,1,REFUND - RF032929,REFUND - RF032929,-2286.3


In [18]:
ProfileReport(df)

Number of variables,14
Number of observations,1501634
Total Missing (%),0.0%
Total size in memory,160.4 MiB
Average record size in memory,112.0 B
Numeric,4
Categorical,10
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# Split Data
Example uses 80-10-10 split for train, eval and test - change if necessary

In [16]:
RANDOM_SEED = 42
train_df = df.sample(frac=0.8, random_state=RANDOM_SEED)
eval_df = df.drop(train_df.index)
test_df = eval_df.sample(frac=0.5, random_state=RANDOM_SEED)
eval_df = eval_df.drop(test_df.index)

In [17]:
def export_datasets(on_cloud=False):
    if on_cloud:
        data_dir = 'gs://{bucket}/spam-classification/data/split'.format(bucket=BUCKET)
    else:
        data_dir = 'data/split'
    
    if not on_cloud:
        if not os.path.exists('data'):
            os.mkdir('data')
        if not os.path.exists('data/split'):
            os.mkdir('data/split')
        
    def export_df(df, filename):
        full_path = os.path.join(data_dir, filename)
        csv_str = '\n'.join(DELIM.join(str(r) for r in rec) for rec in df.to_records(index=False))
        with open(full_path, 'w') as f:
            f.write(csv_str)
    
    export_df(train_df, 'train.csv')
    export_df(eval_df, 'eval.csv')
    export_df(test_df, 'test.csv')
    
    if not os.path.exists('data/misc'):
        os.mkdir('data/misc')
    with open('./data/misc/labels.txt', 'w') as f:
        label_vocab = DELIM.join(list(df[LABEL_COL].astype('str').unique()))
        f.write(label_vocab)
  
    return
  
export_datasets(on_cloud=False)